In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools

In [2]:
function intersection(line1,line2)
    x1,y1,x2,y2 = vcat(line1...)
    x3,y3,x4,y4 = vcat(line2...)

    det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
    if det != 0.0
        a = 1/det
        b = [y1-y2 x2-x1; y3-y4 x4-x3]  # x1-x2 => x2-x1 bug in the source link !!
        c = [x1-x3; y1-y3]
        (β,α) = a * b * c
    else
        if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) # segments collinear
             return nothing
        else
             # segments parallel: no intersection
             return nothing
        end
    end
    return α,β
end

intersection (generic function with 1 method)

In [3]:
line1 = [0.0 0.0; 0.0 1.0]
line2 = [1.0 1.0; 0.0 1.0]
@btime intersection(line1, line2)

  884.265 ns (25 allocations: 592 bytes)


## Controllo se la funzione è type unstable

In [4]:
@code_warntype intersection(line1, line2)

Variables
  #self#::Core.Compiler.Const(intersection, false)
  line1::Array{Float64,2}
  line2::Array{Float64,2}
  @_4::Int64
  @_5::Int64
  @_6::Any
  x1::Any
  y1::Any
  x2::Any
  y2::Any
  x3::Any
  y3::Any
  x4::Any
  y4::Any
  det::Any
  a::Any
  b::Any
  c::Any
  β::Any
  α::Any
  @_21::Any

Body::Union{Nothing, Tuple{Any,Any}}
1 ─       Core.NewvarNode(:(@_6))
│         Core.NewvarNode(:(a))
│         Core.NewvarNode(:(b))
│         Core.NewvarNode(:(c))
│         Core.NewvarNode(:(β))
│         Core.NewvarNode(:(α))
│   %7  = Core._apply_iterate(Base.iterate, Main.vcat, line1)::Union{Array{Any,1}, Array{Float64,1}}
│   %8  = Base.indexed_iterate(%7, 1)::Tuple{Any,Int64}
│         (x1 = Core.getfield(%8, 1))
│         (@_4 = Core.getfield(%8, 2))
│   %11 = Base.indexed_iterate(%7, 2, @_4)::Tuple{Any,Int64}
│         (y1 = Core.getfield(%11, 1))
│         (@_4 = Core.getfield(%11, 2))
│   %14 = Base.indexed_iterate(%7, 3, @_4)::Tuple{Any,Int64}
│         (x2 = Core.getfield(%14, 

la funzione è type unstable in quanto ho nell'output la stringa: 

    Body::Union{Nothing, Tuple{Any,Any}}
    
Il problema è che la funzione può ritornare una tupla di Float64 oppure nothing a seconda se entro nel then oppure nell'else. Il problema è stato risolto sostituendo nothing con ().